#### Actions
Actions are more sofisticated than services used for the same purpose of immediate jobs to be done on request and give response to the requested node. But Actions are asynchronous

While services are handy for simple get/set interactions like querying status and managing configuration, they don’t work well when you need to initiate a long-running task. For example, imagine commanding a robot to drive to some distant location; call it goto_position.

Using an action interface to goto_position, you send a goal, then move on to other tasks while the robot is driving. Along the way, you receive periodic progress updates (distance traveled, estimated time to goal, etc.), culminating in a result message (did the robot make it to the goal or was it forced to give up?). And if something more important comes up, you

### 1. First define the goal, result and feedback of the action in .action file and build it with catkin_make.
### 2. Write the code which do the action and implement it using SimpleActionServer Class from actionlib package. 

In [ ]:
# 1. Defining an Action

#Step 1 - define goal, result and feedback message formats in an action definition file (x.action)

#Example :- realizing timer in an action.

# This is an action definition file, which has three parts: the goal, the result, and the feedback. 
# 
# Part 1: the goal, to be sent by the client 
# 
# The amount of time we want to wait 
duration time_to_wait 
--- 
# Part 2: the result, to be sent by the server upon completion 
#
# How much time we waited 
duration time_elapsed 
# How many updates we provided along the way
uint32 updates_sent 
---
# Part 3: the feedback, to be sent periodically by the server during 
# execution. 
# 
# The amount of time that has elapsed from the start 
duration time_elapsed 
# The amount of time remaining until we're done 
duration time_remaining

# --- are used as seperators between sections goal, result and feedback



\]
Step 2
The above written action file Timer.action should be placed in directory called action with in ROS package (basic).

Step 3
Add actionlib_msgs in find_package   in CMakeLists.txt which helps catkin to build the action in folder

find_package(catkin REQUIRED COMPONENTS # other packages are already listed here actionlib_msgs)

Step 4

add_action_files() in CMakeLists.txt too

add_action_files( DIRECTORY action FILES Timer.action )

Step 5

list the dependencies for your actions. You also need to explicitly list actionlib_msgs as a dependency in order for actions to compile properly:

generate_messages( DEPENDENCIES actionlib_msgs std_msgs)

Step 6
add actionlib_msgs as a dependency for catkin:

catkin_package( CATKIN_DEPENDS actionlib_msgs)

Step 7
edit package.xml by adding

<build_depend>actionlib_msgs</build_depend>
<exec_depend>actionlib_msgs</exec_depend>



Step 8
run catkin_make in ros_ws
and then source devel/setup.bash to update files






#### Write and Build an action server using SimpleActionServer class from the actionlib package.

In [ ]:
# Action server to execute timer action simple_action_server.py
#! /usr/bin/env python 
import rospy
import time 
import actionlib 
from basics.msg import TimerAction, TimerGoal, TimerResult

def do_timer(goal): 
    start_time = time.time() 
    time.sleep(goal.time_to_wait.to_sec()) 
    result = TimerResult() 
    result.time_elapsed = rospy.Duration.from_sec(time.time()- start_time)
    result.updates_sent = 0 
    server.set_succeeded(result)

rospy.init_node('timer_action_server') 
server = actionlib.SimpleActionServer('timer', TimerAction, do_timer, False)   # pass False to disable autostart which is important to avoid bugs
server.start() 
rospy.spin()



Run action

user@hostname$ rosrun basics simple_action_server.py


In [ ]:
#### Use Action simple_action_client.py

import actionlib 
from basics.msg 
import TimerAction, TimerGoal, TimerResult
rospy.init_node('timer_action_client') 
client = actionlib.SimpleActionClient('timer', TimerAction) client.wait_for_server() 
goal = TimerGoal() 
goal.time_to_wait = rospy.Duration.from_sec(5.0)
client.send_goal(goal) 
client.wait_for_result() 
print('Time elapsed: %f'%(client.get_result().time_elapsed.to_sec()))


#### Run simple action client calss to send goal to our action server and wait for result

In [ ]:
#! /usr/bin/env python 
import rospy
import actionlib 
from basics.msg 
import TimerAction, TimerGoal, TimerResult
rospy.init_node('timer_action_client') 
client = actionlib.SimpleActionClient('timer', TimerAction) 
client.wait_for_server() goal = TimerGoal() goal.time_to_wait = rospy.Duration.from_sec(5.0) client.send_goal(goal) 
client.wait_for_result() 
print('Time elapsed: %f'%(client.get_result().time_elapsed.to_sec()))

#### Action server with feedbac and emergency stop facilities

In [ ]:
#! /usr/bin/env python 
import rospy
import time 
import actionlib from basics.msg 
import TimerAction, TimerGoal, TimerResult, TimerFeedback

def do_timer(goal): 
    start_time = time.time() 
    update_count = 0
    if goal.time_to_wait.to_sec() > 60.0:
        result = TimerResult()
        result.time_elapsed = rospy.Duration.from_sec(time.time() - start_time) 
        result.updates_sent = update_count 
        server.set_aborted(result, "Timer aborted due to too-long wait") 
        return
    while (time.time() - start_time) < goal.time_to_wait.to_sec():
        if server.is_preempt_requested():
            result = TimerResult()
            result.time_elapsed = rospy.Duration.from_sec(time.time() - start_time)
            result.updates_sent = update_count
            server.set_preempted(result, "Timer preempted") 
            return
        feedback = TimerFeedback() 
        feedback.time_elapsed = rospy.Duration.from_sec(time.time() - start_time) 
        feedback.time_remaining = goal.time_to_wait - feedback.time_elapsed 
        server.publish_feedback(feedback) 
        update_count += 1
        time.sleep(1.0)
    result = TimerResult() 
    result.time_elapsed = rospy.Duration.from_sec(time.time() - start_time) 
    result.updates_sent = update_count 
    server.set_succeeded(result, "Timer completed successfully")
rospy.init_node('timer_action_server') 
server = actionlib.SimpleActionServer('timer', TimerAction, do_timer, False) 
server.start() 
rospy.spin()

#### Code explanation

To provide feedback, we need to import message type TimerFeedback from basic.msg

We abort action under special cases and for wrong input parameters with sets_aborted()

We include result with set_succeeded() if possible

Action handles the preemptions(premature interruptions) to give status about progress of action and also to address change of goals in runtime.
This can be one using server.is_preempt_requested():
we fill the succeeding result with feedback using server.set_preempted

We include periodic feedback process in action, where the status about progress of action will be published on server using publish_feedback()

once goal is acheived we notify the client. Here we may fill in the updates_sent field and add a status string like "goal completed" using server.set_succeeded(result, "goal completed")




#### advanced client code to test feedback, preempt a goal and trigger an abort.


In [ ]:

#! /usr/bin/env python 
import rospy
import time 
import actionlib from basics.msg 
import TimerAction, TimerGoal, TimerResult, TimerFeedback
def feedback_cb(feedback): 
    print('[Feedback] Time elapsed: %f'%(feedback.time_elapsed.to_sec())) 
    print('[Feedback] Time remaining: %f'%(feedback.time_remaining.to_sec()))

rospy.init_node('timer_action_client') 
client = actionlib.SimpleActionClient('timer', TimerAction)
client.wait_for_server()

goal = TimerGoal() 
goal.time_to_wait = rospy.Duration.from_sec(5.0) # Uncomment this line to test server-side abort: 
#goal.time_to_wait = rospy.Duration.from_sec(500.0)
client.send_goal(goal, feedback_cb=feedback_cb)

# Uncomment these lines to test goal preemption: 
#time.sleep(3.0) 
#client.cancel_goal()
client.wait_for_result() 
print('[Result] State: %d'%(client.get_state())) 
print('[Result] Status: %s'%(client.get_goal_status_text())) print('[Result] Time elapsed: %f'%(client.get_result().time_elapsed.to_sec())) 
print('[Result] Updates sent: %d'%(client.get_result().updates_sent))

#### Summary of topics, services and actions

| Type | Best used for                                     |
|------|---------------------------------------------------|
|Topic | One-way communication, especially if there might be multple nodes listening (e.g., streams of sensor data)|
|------|---------------------------------------------------|
|Service| Simple request/response interactions, such as asking a question about a node's current sate|
|------|---------------------------------------------------|
|Action|Most request/response interactions, especially when servicing the request is no instantaneous(e.g., navigating to a goad location)|